In [2]:
import pandas as pd

# Load dataset
df = pd.read_excel(r"C:\Users\Eeksha\Desktop\GSECD-final-project1.0\version2\civic-data-forge-main\supabase\data\dataset2.xlsx")

# ----------------------------
# Define crisis conditions with research-backed thresholds
# ----------------------------
conditions = pd.DataFrame({
    "low_yield": df["Cereal yield (kg per hectare)"] < 1000,                # global concern minimum
    "high_imports": df["Food imports (% of merchandise imports)"] > 15,     # import stress threshold
    "low_production": df["Food production index (2014-2016 = 100)"] < 100,  # early warning
    "low_gdp_growth": df["GDP growth (annual %)"] < 2,                      # economic stress
    "low_gdp_per_capita": df["GDP per capita (current US$)"] < 784,         # $2.15/day poverty line
    "high_inflation": df["Inflation, consumer prices (annual %)"] > 10,     # risk threshold
    "high_population_growth": df["Population growth (annual %)"] > 2.5       # food system pressure
})

# ----------------------------
# Count how many conditions are True per row
# ----------------------------
df["Crisis_Count"] = conditions.sum(axis=1)

# ----------------------------
# Label: Crisis (1) if 3 or more risk signals, else No Crisis (0)
# ----------------------------
df["Crisis_Label"] = (df["Crisis_Count"] >= 3).astype(int)

# ----------------------------
# Save updated dataset (optional)
# ----------------------------
df.to_csv("food_crisis_dataset_labeled.csv", index=False)

print("✅ Crisis labels (0/1) added successfully!")
print(df["Crisis_Label"].value_counts())


✅ Crisis labels (0/1) added successfully!
Crisis_Label
0    3432
1    1194
Name: count, dtype: int64


In [3]:
# food_crisis_workflow.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import shap
import warnings
warnings.filterwarnings("ignore")

# ===============================
# 1. Load dataset
# ===============================
# Update path if needed
data = pd.read_csv(r"C:\Users\Eeksha\Desktop\GSECD-final-project1.0\version2\civic-data-forge-main\supabase\models\food_crisis_dataset_labeled.csv")

# Ensure Year column is int (same idea as your economic code's Date)
if "Year" in data.columns:
    data["Year"] = data["Year"].astype(int)
else:
    raise KeyError("Expected 'Year' column in dataset.")

# ===============================
# 2. Features and target
# ===============================
# Drop the label + identifying cols (adjust names if differ)
drop_cols = ["Crisis_Label", "Country Name", "Year", "Country Code", "Crisis_Count"]
X = data.drop(columns=[c for c in drop_cols if c in data.columns], errors="ignore")
y = data["Crisis_Label"]
country_year = data[["Country Name", "Year"]]

# (Optional) keep the feature order consistent (you can adjust list if necessary)
# expected_features = [...]
# X = X[expected_features]

# ===============================
# 3. Split dataset (train/test)
# ===============================
X_train, X_test, y_train, y_test, cy_train, cy_test = train_test_split(
    X, y, country_year, test_size=0.2, random_state=42, stratify=y
)

# ===============================
# 4. Train Random Forest (same pattern as economic code)
# ===============================
model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
model.fit(X_train, y_train)

# Evaluate accuracy + ROC-AUC (print like your economic model)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Model Accuracy: {accuracy * 100:.2f}%\n")

# ROC-AUC if probabilities available and both classes present in y_test
if y_prob is not None and len(np.unique(y_test)) > 1:
    try:
        auc = roc_auc_score(y_test, y_prob)
        print(f"✅ ROC-AUC Score: {auc:.3f}\n")
    except Exception:
        pass

# ===============================
# 5. Predict for Next Year (simulate next year like economic code)
# ===============================
latest_year = data["Year"].max()
future_data = data[data["Year"] == latest_year].copy()
future_data["Year"] = latest_year + 1  # simulate next year (predict for Year+1)

# Build X_future by dropping same columns
X_future = future_data.drop(columns=[c for c in drop_cols if c in future_data.columns], errors="ignore")

# If X_future empty -> nothing to predict
if X_future.shape[0] == 0:
    print("🌍 No countries available for prediction (check latest year rows).")
else:
    if hasattr(model, "predict_proba"):
        future_probs = model.predict_proba(X_future)[:, 1]
    else:
        future_probs = model.predict(X_future)  # fallback, 0/1

    future_data["Crisis_Probability"] = future_probs
    # keep probability in [0,1]; if previously not, ensure scaling below if needed

    # ===============================
    # 6. Identify High-Risk Countries
    # ===============================
    threshold = 0.80  # show countries with ≥80% crisis probability
    high_risk = future_data[future_data["Crisis_Probability"] >= threshold]

    if high_risk.empty:
        print("🌍 No countries exceed the crisis probability threshold.")
    else:
        # SHAP explanation - compute shap on X_future, align with rows
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_future)  # may be list or array

        # Select SHAP values for positive class (class 1) robustly
        if isinstance(shap_values, list) and len(shap_values) > 1:
            shap_pos = shap_values[1]
        else:
            shap_pos = shap_values

        # shap_pos may sometimes be 3D depending on shap version; normalize to 2D (n_samples, n_features)
        if hasattr(shap_pos, "ndim") and shap_pos.ndim == 3:
            # common shape: (n_samples, n_features, n_classes) -> take class 1
            shap_pos = shap_pos[:, :, 1]

        # convert to DataFrame aligned with X_future index
        shap_df = pd.DataFrame(shap_pos, columns=X_future.columns, index=X_future.index)

        print("\n🔥 High-Risk Countries (≥80% Crisis Probability)\n")
        for idx in high_risk.index:
            country = high_risk.loc[idx, "Country Name"]
            prob = high_risk.loc[idx, "Crisis_Probability"] * 100 if future_data["Crisis_Probability"].max() <= 1 else high_risk.loc[idx, "Crisis_Probability"] * 100
            # If probabilities are 0-1 -> multiply by 100, if they are already 0-100 you'd adjust; here we assume 0-1.

            # Get SHAP row for this index (aligned)
            if idx in shap_df.index:
                instance_shap = shap_df.loc[idx].values
            else:
                # fallback: find by position
                instance_shap = shap_df.iloc[list(high_risk.index).index(idx)].values

            # Pair features with shap values and feature value (from X_future)
            feature_contribs = sorted(
                zip(X_future.columns, instance_shap, X_future.loc[idx].values),
                key=lambda x: abs(x[1]),
                reverse=True
            )[:3]

            print(f"🚨 {country} — Crisis Probability: {prob:.2f}%")
            for feat, contrib, val in feature_contribs:
                print(f"   {feat:<45} Contribution: {contrib:+.3f} | Value: {val:.2f}")
            print("-" * 70)

    # Save predictions for the simulated next year
    out_file = "food_future_predictions.csv"
    future_data.to_csv(out_file, index=False)
    print(f"\n📂 Saved future-year predictions to: {out_file}")


c:\Users\Eeksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✅ Model Accuracy: 96.87%

✅ ROC-AUC Score: 0.997


🔥 High-Risk Countries (≥80% Crisis Probability)

🚨 Botswana — Crisis Probability: 95.00%
   Food imports (% of merchandise imports)       Contribution: +0.193 | Value: 16.21
   Cereal yield (kg per hectare)                 Contribution: +0.179 | Value: 468.90
   GDP growth (annual %)                         Contribution: +0.119 | Value: -2.99
----------------------------------------------------------------------
🚨 Burundi — Crisis Probability: 91.00%
   GDP per capita (current US$)                  Contribution: +0.283 | Value: 153.93
   Inflation, consumer prices (annual %)         Contribution: +0.147 | Value: 20.21
   Population growth (annual %)                  Contribution: +0.129 | Value: 2.58
----------------------------------------------------------------------
🚨 Cabo Verde — Crisis Probability: 97.50%
   Cereal yield (kg per hectare)                 Contribution: +0.351 | Value: 23.00
   Food imports (% of merchandise import

In [7]:
# # food_ews_timeaware.py
# # random forest model
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import (
#     accuracy_score, precision_score, recall_score, f1_score,
#     confusion_matrix, roc_auc_score, classification_report, average_precision_score
# )
# import shap
# import warnings
# warnings.filterwarnings("ignore")
# RANDOM_STATE = 42

# # ------------------------
# # Config / file paths
# # ------------------------
# INPUT_FILE = "/content/food_crisis_dataset_labeled.csv"  # <- change to your file
# OUT_PRED_CSV = "food_crisis_2025_predictions.csv"
# EVAL_FEATURE_YEAR = 2023   # features year used for evaluation (to predict EVAL_FEATURE_YEAR+1)
# PREDICT_FROM_YEAR = 2024   # year whose features we will use to predict 2025
# PROB_THRESHOLD = 0.70      # threshold for flagging high risk (0-1)

# # ------------------------
# # 1) Load data
# # ------------------------
# df = pd.read_csv(INPUT_FILE)
# df = df.copy()
# if "Year" not in df.columns:
#     raise KeyError("Dataset must contain 'Year' column.")
# df["Year"] = df["Year"].astype(int)

# required_cols = ["Country Name", "Year", "Crisis_Label"]
# if not all(c in df.columns for c in required_cols):
#     raise KeyError(f"Dataset must contain columns: {required_cols}")

# # Choose features (ensure names match your CSV)
# features = [
#     "Cereal yield (kg per hectare)",
#     "Food imports (% of merchandise imports)",
#     "Food production index (2014-2016 = 100)",
#     "GDP (current US$)",
#     "GDP growth (annual %)",
#     "GDP per capita (current US$)",
#     "Inflation, consumer prices (annual %)",
#     "Population growth (annual %)"
# ]

# # Validate features present
# missing_feats = [c for c in features if c not in df.columns]
# if missing_feats:
#     raise KeyError(f"Missing feature columns in CSV: {missing_feats}")

# # ------------------------
# # 2) Build forecasting target: Crisis at t+1
# #    For each country, target = Crisis_Label shifted -1 (next year)
# # ------------------------
# df = df.sort_values(["Country Name", "Year"]).reset_index(drop=True)
# df["Crisis_Target_t1"] = df.groupby("Country Name")["Crisis_Label"].shift(-1)

# # Drop rows where target is NaN (i.e., last year for each country)
# df_model = df.dropna(subset=features + ["Crisis_Target_t1"]).copy()
# df_model["Crisis_Target_t1"] = df_model["Crisis_Target_t1"].astype(int)

# # ------------------------
# # 3) Prepare evaluation split (time-aware)
# #    Train on years <= EVAL_FEATURE_YEAR-1  (features up to 2022)
# #    Test on rows where Year == EVAL_FEATURE_YEAR (features at 2023, target is 2024)
# # ------------------------
# train_cutoff = EVAL_FEATURE_YEAR - 1  # e.g., 2022
# train_rows = df_model[df_model["Year"] <= train_cutoff].copy()
# test_rows = df_model[df_model["Year"] == EVAL_FEATURE_YEAR].copy()  # features=2023 -> target=2024

# if train_rows.empty or test_rows.empty:
#     raise ValueError(f"Not enough data for train/test with EVAL_FEATURE_YEAR={EVAL_FEATURE_YEAR}.")

# X_train = train_rows[features].values
# y_train = train_rows["Crisis_Target_t1"].values.astype(int)
# X_test = test_rows[features].values
# y_test = test_rows["Crisis_Target_t1"].values.astype(int)

# # Impute missing values (median)
# imputer = SimpleImputer(strategy="median")
# X_train = imputer.fit_transform(X_train)
# X_test = imputer.transform(X_test)

# # ------------------------
# # 4) Train model (Random Forest)
# # ------------------------
# model = RandomForestClassifier(
#     n_estimators=300,
#     random_state=RANDOM_STATE,
#     class_weight="balanced_subsample",
#     n_jobs=-1
# )
# model.fit(X_train, y_train)

# # ------------------------
# # 5) Evaluate on test (features=EVAL_FEATURE_YEAR predict EVAL_FEATURE_YEAR+1)
# # ------------------------
# y_pred = model.predict(X_test)
# y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

# acc = accuracy_score(y_test, y_pred)
# prec = precision_score(y_test, y_pred, zero_division=0)
# rec = recall_score(y_test, y_pred, zero_division=0)
# f1 = f1_score(y_test, y_pred, zero_division=0)
# cm = confusion_matrix(y_test, y_pred)
# roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None and len(np.unique(y_test)) > 1 else float("nan")
# pr_auc = average_precision_score(y_test, y_prob) if y_prob is not None and len(np.unique(y_test)) > 1 else float("nan")

# print("\n=== Evaluation (simulating prediction of year", EVAL_FEATURE_YEAR+1, ") ===")
# print(f"Training years used (features -> target): {train_rows['Year'].min()}-{train_rows['Year'].max()} -> targets {train_rows['Year'].min()+1}-{train_rows['Year'].max()+1}")
# print(f"Test year (features): {EVAL_FEATURE_YEAR} -> target year: {EVAL_FEATURE_YEAR+1}\n")

# print(f"Accuracy : {acc:.4f}")
# print(f"Precision: {prec:.4f}")
# print(f"Recall   : {rec:.4f}")
# print(f"F1-score : {f1:.4f}")
# print(f"ROC-AUC  : {roc_auc:.4f}")
# print(f"PR-AUC   : {pr_auc:.4f}")
# print("\nConfusion Matrix (rows=actual target, cols=predicted):")
# print(cm)
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, zero_division=0))

# # ------------------------
# # 6) Retrain on all available up to Year == EVAL_FEATURE_YEAR (i.e., use features through 2023 to learn targets through 2024)
# #    Then use rows where Year == PREDICT_FROM_YEAR (2024) to predict 2025
# # ------------------------
# train_full_rows = df_model[df_model["Year"] <= EVAL_FEATURE_YEAR].copy()  # rows with features <=2023 and targets up to 2024
# X_full = train_full_rows[features].values
# y_full = train_full_rows["Crisis_Target_t1"].values.astype(int)
# X_full = imputer.fit_transform(X_full)  # refit imputer on full training
# model.fit(X_full, y_full)               # retrain on full

# # Prepare 2024 features to predict 2025
# predict_feat_rows = df[df["Year"] == PREDICT_FROM_YEAR].copy()
# if predict_feat_rows.empty:
#     raise ValueError(f"No rows found for Year == {PREDICT_FROM_YEAR} to use as features for predicting {PREDICT_FROM_YEAR+1}.")

# X_predict = predict_feat_rows[features].values
# X_predict = imputer.transform(X_predict)
# probs_2025 = model.predict_proba(X_predict)[:, 1]
# predict_feat_rows = predict_feat_rows.reset_index(drop=True)
# predict_feat_rows["Crisis_Probability_2025"] = probs_2025
# predict_feat_rows["Crisis_Pred_2025"] = (predict_feat_rows["Crisis_Probability_2025"] >= PROB_THRESHOLD).astype(int)

# # ------------------------
# # 7) SHAP explanations for the predicted rows (2025)
# # ------------------------
# explainer = shap.TreeExplainer(model)
# shap_values_all = explainer.shap_values(X_predict)

# # pick positive class shap values robustly
# if isinstance(shap_values_all, list) and len(shap_values_all) > 1:
#     shap_pos = shap_values_all[1]
# else:
#     shap_pos = shap_values_all

# # ensure 2D: (n_samples, n_features)
# if hasattr(shap_pos, "ndim") and shap_pos.ndim == 3:
#     # some versions return shape (n_samples, n_features, n_classes)
#     shap_pos = shap_pos[:, :, 1]

# shap_df = pd.DataFrame(shap_pos, columns=features, index=predict_feat_rows.index)

# # ------------------------
# # 8) Display 2025 predictions and SHAP contributions
# # ------------------------
# print(f"\n=== 2025 Early-Warning Predictions (based on {PREDICT_FROM_YEAR} features) ===")
# # show top countries by probability
# predict_feat_rows["Crisis_Probability_2025_pct"] = predict_feat_rows["Crisis_Probability_2025"] * 100
# pred_sorted = predict_feat_rows.sort_values("Crisis_Probability_2025", ascending=False)

# # Print summary top N (all or filtered)
# topN = 50
# display_df = pred_sorted[["Country Name", "Country Code", "Crisis_Probability_2025", "Crisis_Pred_2025"]].head(topN).copy()
# display_df["Crisis_Probability_2025"] = (display_df["Crisis_Probability_2025"]*100).round(2)
# print(display_df.to_string(index=False))

# # Print high-risk countries with SHAP details
# high_risk = predict_feat_rows[predict_feat_rows["Crisis_Probability_2025"] >= PROB_THRESHOLD]
# if high_risk.empty:
#     print(f"\nNo countries exceed the threshold of {PROB_THRESHOLD:.2f} probability for 2025.")
# else:
#     print(f"\n🔥 High-risk countries (prob >= {PROB_THRESHOLD:.2f}) with top 3 SHAP contributors:\n")
#     for idx in high_risk.index:
#         row = predict_feat_rows.loc[idx]
#         country = row["Country Name"]
#         prob = row["Crisis_Probability_2025"] * 100
#         print(f"🚨 {country} — Predicted Crisis Probability 2025: {prob:.2f}%")
#         # get SHAP for this index (shap_df index aligns with predict_feat_rows.reset_index)
#         s = shap_df.loc[idx]
#         top_feats = s.abs().sort_values(ascending=False).index[:3]
#         for feat in top_feats:
#             contrib = s[feat]
#             val = row[feat]
#             sign = "+" if contrib > 0 else "-"
#             print(f"   {feat:<50} Contribution: {contrib:+.4f} | Value: {val}")
#         print("-"*80)

# # ------------------------
# # 9) Save predictions to CSV
# # ------------------------
# out_cols = ["Country Name", "Country Code", "Year"] + features + ["Crisis_Probability_2025", "Crisis_Pred_2025"]
# predict_feat_rows[out_cols].to_csv(OUT_PRED_CSV, index=False)
# print(f"\n✅ 2025 predictions saved to: {OUT_PRED_CSV}\n")


In [4]:
import joblib

joblib.dump(model, 'food_crisis_model.pkl')

['food_crisis_model.pkl']